In [1]:
import pathlib

model_data_path = pathlib.Path("model_data").resolve()
db_path = pathlib.Path("../staffing.db").resolve()

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(f"sqlite:///{db_path}")

In [3]:
query_assignments = """
SELECT
  name, start_time, end_time
FROM assignments
INNER JOIN staff on staff.id == assignments.staff_id
INNER JOIN shifts on shifts.id == assignments.shift_id
"""

query_uncovered = """
SELECT
  id, start_time, end_time, IfNull(assigned, 0) as assigned, staff_count as required, qualification_id
FROM (
    SELECT
    shift_id, count(staff_id) as assigned
    FROM assignments
    GROUP BY shift_id
)
RIGHT JOIN shifts on shifts.id == shift_id
"""

with engine.connect() as connection:
    assignments = pd.read_feather(model_data_path / "assignments.feather")
    assignments.to_sql("assignments", connection, if_exists="replace", index_label="id")
    assignments = pd.read_sql(query_assignments, connection, parse_dates=["start_time", "end_time"]).sample(5)
    uncovered = pd.read_sql(query_uncovered, connection, parse_dates=["start_time", "end_time"]).query("assigned < required")

In [4]:
assignments.head()

,name,start_time,end_time
44,Vincent,2023-05-05 08:00:00,2023-05-05 16:00:00
52,Pauline,2023-05-07 08:00:00,2023-05-07 16:00:00
36,Femke,2023-05-11 08:00:00,2023-05-11 16:00:00
33,Femke,2023-05-01 08:00:00,2023-05-01 16:00:00
24,Matsumi,2023-05-04 08:00:00,2023-05-04 16:00:00


In [5]:
uncovered

,id,start_time,end_time,assigned,required,qualification_id
11,11,2023-05-12 08:00:00,2023-05-12 16:00:00,4,5,1
12,12,2023-05-13 08:00:00,2023-05-13 16:00:00,4,5,1
13,13,2023-05-14 08:00:00,2023-05-14 16:00:00,4,5,1
25,25,2023-05-12 08:00:00,2023-05-12 16:00:00,0,1,0
26,26,2023-05-13 08:00:00,2023-05-13 16:00:00,0,1,0
27,27,2023-05-14 08:00:00,2023-05-14 16:00:00,0,1,0
